In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
import numpy as np
import datetime
import itertools
from keras import regularizers

Using TensorFlow backend.


In [3]:
df = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

mapping_ord1 = {'Unknown': 0, 'Novice': 1, 'Expert': 2, 'Contributor': 3, 'Master': 4, 'Grandmaster': 5}
mapping_ord2 = {'Unknown': 0, 'Freezing': 1, 'Cold': 2, 'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}
mapping_ord3 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_3'].fillna("0"))))])
mapping_ord4 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_4'].fillna("0"))))])
mapping_ord5 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_5'].fillna("0"))))])

def preprocess_data(df):
    df['ord_0'] = df['ord_0'].fillna(0)
    df['ord_1'] =  df['ord_1'].fillna('Unknown').map(mapping_ord1)
    df['ord_2'] =  df['ord_2'].fillna('Unknown').map(mapping_ord2)
    df['ord_3'] =  df['ord_3'].fillna('0').map(mapping_ord3)
    df['ord_4'] =  df['ord_4'].fillna('0').map(mapping_ord4)
    df['ord_5'] =  df['ord_5'].fillna('0').map(mapping_ord5)
    df['bin_3'] = df['bin_3'].fillna('U').map({"T": 1, "F": 0, "U": np.nan})
    df['bin_4'] = df['bin_4'].fillna('U').map({"Y": 1, "N": 0, "U": np.nan})
    return df

def encode(df, df_test, cols):
    cols_enc = list(map(lambda x: x + "_enc", cols))
    for c in cols_enc:
        df[c] = np.nan
        df_test[c] = np.nan
        
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    for train_idx, oof_idx in tqdm(skf.split(df, df["target"])):
        enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
        enc.fit(df.loc[train_idx, cols], df.loc[train_idx, "target"])
        df.loc[oof_idx, cols_enc] = enc.transform(df.loc[oof_idx, cols]).values
    
    enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
    enc.fit(df[cols], df["target"])
    df_test[cols_enc] = enc.transform(df_test[cols])
    return df, df_test, cols_enc
    
df = preprocess_data(df)
df_test = preprocess_data(df_test)

binary = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
ordinal  = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
low_card = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
high_card = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
date = ['day', 'month']

features = high_card + low_card + date # + ordinal + binary

df, df_test, features_enc =  encode(df, df_test, features)

for c in features:
    enc = preprocessing.LabelEncoder()
    enc.fit(pd.concat([df, df_test], sort=False)[c].fillna(-1).astype(str))
    df[c] = enc.transform(df[c].fillna(-1).astype(str))
    df_test[c] = enc.transform(df_test[c].fillna(-1).astype(str))
    
features_all = features  + features_enc

5it [00:39,  7.89s/it]


In [15]:
def create_model(df, cols_te, cols_emb):    
    
    inputs = []
    outputs = []
    
    reg = regularizers.l1_l2(l1=1e-5, l2=1e-5)
    
    inp_dense = layers.Input(shape=(len(cols_te),))
    x_dense = layers.Dense(32, activation="relu")(inp_dense)
    x_dense = layers.Dropout(0.3)(x_dense)
    out_dense = layers.BatchNormalization()(x_dense)
    
    inputs.append(inp_dense)
    outputs.append(out_dense)
    
    for c in cols_emb:
        nunique = int(df[c].nunique())
        emb_dim = int(min(np.ceil((nunique)/2), 64))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(nunique + 1, emb_dim, name=c, embeddings_regularizer=reg)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
            
        out = layers.Reshape(target_shape=(emb_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate(name="embeddings")(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    
    return model

In [16]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
def get_layer_output(X, model):
    layer = Model(inputs=model.input, outputs=model.get_layer("embeddings").output)
    out = emd_layer.predict(X)
    return out

In [17]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

folds = 10
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

es = EarlyStopping(monitor='val_auc', min_delta=0.0001, patience=5, verbose=1, mode='max',  
                   baseline=None,  restore_best_weights=True)

rlr = ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, mode='max', verbose=1)

df_test["target"] = 0
X_test = [df_test.loc[:, f].values for f in features]

df_test["target"] = 0

scaller = StandardScaler()

for train_idx, val_idx in tqdm(skf.split(df, df["target"])):
       
    X_train = [scaller.fit_transform(df.loc[train_idx, features_enc].values)]
    X_train += [df.loc[train_idx, f].values for f in features]
    y_train = df.loc[train_idx, "target"].values
    
    X_val = [scaller.transform(df.loc[val_idx, features_enc].values)] 
    X_val += [df.loc[val_idx, f].values for f in features]
    y_val = df.loc[val_idx, "target"].values
    
    model = create_model(df, features_enc, features)
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=[auc])

    model.fit(X_train, utils.to_categorical(y_train), validation_data=(X_val, utils.to_categorical(y_val)),
              verbose=1, batch_size=1024, callbacks=[es, rlr], epochs=100)
       
    X_test = [scaller.transform(df_test.loc[:, features_enc].values)] 
    X_test += [df_test.loc[:, f].values for f in features]
    df_test["target"] += model.predict(X_test)[:, 1]
 
    K.clear_session()


0it [00:00, ?it/s]

Train on 540000 samples, validate on 60000 samples
Epoch 1/100


/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


540000/540000 [==============================] - 16s 30us/sample - loss: 0.5568 - auc: 0.7029 - val_loss: 0.4665 - val_auc: 0.7831
Epoch 2/100
540000/540000 [==============================] - 9s 17us/sample - loss: 0.4550 - auc: 0.7759 - val_loss: 0.4367 - val_auc: 0.7838
Epoch 3/100
540000/540000 [==============================] - 9s 16us/sample - loss: 0.4315 - auc: 0.7815 - val_loss: 0.4237 - val_auc: 0.7833
Epoch 4/100
540000/540000 [==============================] - 9s 16us/sample - loss: 0.4227 - auc: 0.7839 - val_loss: 0.4195 - val_auc: 0.7834
Epoch 5/100
539648/540000 [============================>.] - ETA: 0s - loss: 0.4204 - auc: 0.7845
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
540000/540000 [==============================] - 9s 16us/sample - loss: 0.4204 - auc: 0.7845 - val_loss: 0.4184 - val_auc: 0.7835
Epoch 6/100
540000/540000 [==============================] - 9s 16us/sample - loss: 0.4140 - auc: 0.7881 - val_loss: 0.4151 - val_auc: 


1it [02:03, 123.49s/it]

...


9it [19:46, 132.94s/it]

Train on 540000 samples, validate on 60000 samples
Epoch 1/100


/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


540000/540000 [==============================] - 13s 25us/sample - loss: 0.5539 - auc: 0.7017 - val_loss: 0.4648 - val_auc: 0.7831
Epoch 2/100
540000/540000 [==============================] - 10s 18us/sample - loss: 0.4537 - auc: 0.7756 - val_loss: 0.4351 - val_auc: 0.7837
Epoch 3/100
540000/540000 [==============================] - 10s 19us/sample - loss: 0.4304 - auc: 0.7814 - val_loss: 0.4224 - val_auc: 0.7840
Epoch 4/100
540000/540000 [==============================] - 11s 20us/sample - loss: 0.4222 - auc: 0.7833 - val_loss: 0.4188 - val_auc: 0.7839
Epoch 5/100
540000/540000 [==============================] - 10s 19us/sample - loss: 0.4199 - auc: 0.7843 - val_loss: 0.4173 - val_auc: 0.7839
Epoch 6/100
538624/540000 [============================>.] - ETA: 0s - loss: 0.4190 - auc: 0.7849
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
540000/540000 [==============================] - 10s 18us/sample - loss: 0.4191 - auc: 0.7849 - val_loss: 0.4175 - val_


10it [21:57, 131.72s/it]


In [17]:
# df_test['target'] /= folds
# df_test[['id', 'target']].to_csv('../data/submission_entity_encoding_2.csv', index=False)
# !head -n 5 ../data/submission_entity_encoding.csv

id,target
600000,0.14275625
600001,0.2633614
600002,0.1735541
600003,0.14033902


# Combine submissions

In [1]:
# import pandas as pd
# df1 = pd.read_csv('../data/submission_entity_encoding_2.csv')
# df2 = pd.read_csv('../data/submission_lgbm.csv')
# df2["target"] = (df1["target"] + df2["target"])/2
# df2.to_csv('../data/submission_combine.csv', index=False)
# !head -n 5 ../data/submission_combine.csv

id,target
600000,0.12703491030357303
600001,0.26469590910102686
600002,0.1769536913258547
600003,0.11936844049592421


In [4]:
!open ../